# Postgresql Database Client examples.

This notebook contains examples on how use the postgres database client on a sample database.

The sample database is the publicly available [Sakila sample database](https://dev.mysql.com/doc/sakila/en/) of MySQL, it can be downloaded for postgres from [here](https://www.postgresqltutorial.com/postgresql-sample-database/).

The `.tar` file with the data is available in the project at `./tests/fixtures/dvdrental.tar`, and you can load it to a postgres database using the `psql` and `pg_restore` tools built into postgresql.

The sample database is called `dvd_rentals` and represents the business processes of a DVD rental store.

First set environment variables to connect to the database:

The environment variable names and their default values can be set in the `conf.py` '`EnvVariablesConf`' class

In [1]:
%env HERO_POSTGRES_DBNAME=dvd_rentals
%env HERO_POSTGRES_USERNAME=hero
%env HERO_POSTGRES_PASSWORD=hero@2020

env: HERO_POSTGRES_DBNAME=dvd_rentals
env: HERO_POSTGRES_USERNAME=hero
env: HERO_POSTGRES_PASSWORD=hero@2020


Import some required modules:

In [2]:
from hero_db_utils.clients import PostgresDatabaseClient
from hero_db_utils.queries.postgres import QueryOp, QueryFunc

import pandas as pd
pd.set_option('display.max_columns', 35)
pd.set_option('display.max_colwidth',500)

from datetime import datetime
import pytz

Intitiate the Database Client:

In [3]:
# Initiate the database from the environment variable values:
conn_kwargs = PostgresDatabaseClient.get_params_from_env_variables()
client = PostgresDatabaseClient(**conn_kwargs, create_database=False)
client

<PostgresDatabaseClient: 'dvd_rentals' DBClient>

In [4]:
# Get count of rows and columns of the tables in the 'public' schema:
client.get_schema_tables().sort_values("nrows", ascending=False)

,table_name,nrows,ncols
16,rental,16044,7
15,payment,14596,6
9,film_actor,5462,3
12,inventory,4581,4
8,film,1000,13
10,film_category,1000,3
11,film_list,997,8
14,nicer_but_slower_film_list,997,8
2,address,603,8
4,city,600,4


In [5]:
# Read all data from a table:
client.select("category")

,category_id,name,last_update
0,1,Action,2006-02-15 09:46:27
1,2,Animation,2006-02-15 09:46:27
2,3,Children,2006-02-15 09:46:27
3,4,Classics,2006-02-15 09:46:27
4,5,Comedy,2006-02-15 09:46:27
5,6,Documentary,2006-02-15 09:46:27
6,7,Drama,2006-02-15 09:46:27
7,8,Family,2006-02-15 09:46:27
8,9,Foreign,2006-02-15 09:46:27
9,10,Games,2006-02-15 09:46:27


Get data from tables using custom parameters for selection, filters, etc.

In [29]:
# Get rentals where the 'rental_date' is between two dates. 
from_date = datetime(year=2005,month=5,day=26,hour=0,minute=0,tzinfo=pytz.utc)
to_date = datetime(year=2005,month=5,day=26,hour=0,minute=30,tzinfo=pytz.utc)
df = client.select(
    "rental",
    projection=["rental_id","inventory_id","customer_id","rental_date","return_date"],
    filters={"rental_date":QueryOp.between(from_date,to_date)},
    order_by=["rental_id"]
)
df

,rental_id,inventory_id,customer_id,rental_date,return_date
0,146,1431,433,2005-05-26 00:07:11,2005-06-04 00:20:11
1,147,633,274,2005-05-26 00:17:50,2005-05-29 23:21:50
2,148,4252,142,2005-05-26 00:25:23,2005-06-01 19:29:23
3,149,1084,319,2005-05-26 00:28:05,2005-06-02 21:30:05
4,150,909,429,2005-05-26 00:28:39,2005-06-01 02:10:39


In [30]:
#  Count of films by rating:
df = client.select(
    "film",
    projection=["rating", QueryFunc.count()],
    order_by=["count"],
    ascending=False,
    group_by=["rating"],
)
df

,rating,count
0,PG-13,223
1,NC-17,210
2,R,195
3,PG,194
4,G,178


We can also build queries and execute them after with `build_query`, it takes the same arguments as `select`:

In [47]:
# Contruct query to get 10 films where the category is 'PG' and the rental rate is greater than '3' dollars ordered by length descending:
query = client.build_query(
    "film",
    projection=["film_id","title","rental_rate","length","rating","language_id"],
    filters={"rating":"PG","rental_rate":QueryOp.greater_than(3)},
    order_by=["length"],
    ascending=False,
    limit=10
) # 'query' is a 'queries.DBQuery' object

# Print sql of resulting query:
print(client.parse_query(query))

SELECT "film_id","title","rental_rate","length","rating","language_id" FROM "film" WHERE ("rating" = 'PG') AND ("rental_rate" > 3) ORDER BY "length" DESC LIMIT 10


In [48]:
# Execute the query:
client.read_query(query)

,film_id,title,rental_rate,length,rating,language_id
0,719,Records Zorro,4.99,182,PG,1
1,591,Monsoon Cause,4.99,182,PG,1
2,88,Born Spinal,4.99,179,PG,1
3,871,Sweden Shining,4.99,176,PG,1
4,359,Gladiator Westward,4.99,173,PG,1
5,902,Trading Pinocchio,4.99,170,PG,1
6,265,Dying Maker,4.99,168,PG,1
7,706,Queen Luke,4.99,163,PG,1
8,74,Birch Antitrust,4.99,162,PG,1
9,327,Fool Mockingbird,4.99,158,PG,1


In [46]:
# Query to get 5 unique film titles:
query = client.build_query(
    "film",
    projection=["title"],
    distinct=True,
    limit=5
)
# Print resulting sql query:
print(client.parse_query(query))
# Execute the query:
client.read_query(query)

SELECT DISTINCT "title" FROM "film" LIMIT 5


,title
0,Academy Dinosaur
1,Ace Goldfinger
2,Adaptation Holes
3,Affair Prejudice
4,African Egg


Make queries with a JOIN:

In [7]:
# Join 'film' with 'language' to get the name of the language of the movies.
query = client.build_query(
    "film",
    projection=[
        QueryFunc.relation("language","name", alias="film_language"),
        QueryFunc.relation("film","*"),
    ],
    join_table="language",
    join_on={QueryFunc.relation("film","language_id"):QueryFunc.relation("language","language_id")},
    order_by=["film_id"],
    limit=5
)
# Print sql query:
print(client.parse_query(query))
# Execute the query:
client.read_query(query)

SELECT "language"."name" AS "film_language","film".* FROM "film" INNER JOIN "language" ON ("film"."language_id" = "language"."language_id") ORDER BY "film_id" ASC LIMIT 5


,film_language,film_id,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,last_update,special_features,fulltext
0,English,1,Academy Dinosaur,A Epic Drama of a Feminist And a Mad Scientist who must Battle a Teacher in The Canadian Rockies,2006,1,6,0.99,86,20.99,PG,2013-05-26 14:50:58.951,"[Deleted Scenes, Behind the Scenes]",'academi':1 'battl':15 'canadian':20 'dinosaur':2 'drama':5 'epic':4 'feminist':8 'mad':11 'must':14 'rocki':21 'scientist':12 'teacher':17
1,English,2,Ace Goldfinger,A Astounding Epistle of a Database Administrator And a Explorer who must Find a Car in Ancient China,2006,1,3,4.99,48,12.99,G,2013-05-26 14:50:58.951,"[Trailers, Deleted Scenes]",'ace':1 'administr':9 'ancient':19 'astound':4 'car':17 'china':20 'databas':8 'epistl':5 'explor':12 'find':15 'goldfing':2 'must':14
2,English,3,Adaptation Holes,A Astounding Reflection of a Lumberjack And a Car who must Sink a Lumberjack in A Baloon Factory,2006,1,7,2.99,50,18.99,NC-17,2013-05-26 14:50:58.951,"[Trailers, Deleted Scenes]","'adapt':1 'astound':4 'baloon':19 'car':11 'factori':20 'hole':2 'lumberjack':8,16 'must':13 'reflect':5 'sink':14"
3,English,4,Affair Prejudice,A Fanciful Documentary of a Frisbee And a Lumberjack who must Chase a Monkey in A Shark Tank,2006,1,5,2.99,117,26.99,G,2013-05-26 14:50:58.951,"[Commentaries, Behind the Scenes]",'affair':1 'chase':14 'documentari':5 'fanci':4 'frisbe':8 'lumberjack':11 'monkey':16 'must':13 'prejudic':2 'shark':19 'tank':20
4,English,5,African Egg,A Fast-Paced Documentary of a Pastry Chef And a Dentist who must Pursue a Forensic Psychologist in The Gulf of Mexico,2006,1,6,2.99,130,22.99,G,2013-05-26 14:50:58.951,[Deleted Scenes],'african':1 'chef':11 'dentist':14 'documentari':7 'egg':2 'fast':5 'fast-pac':4 'forens':19 'gulf':23 'mexico':25 'must':16 'pace':6 'pastri':10 'psychologist':20 'pursu':17


Join multiple tables:

In [11]:
# Join multiple tables to get the full address of stores:
query = client.build_query(
    "store",
    projection=[
        QueryFunc.relation("store","store_id"),
        QueryFunc.relation("country","country"),
        QueryFunc.relation("city","city"),
        QueryFunc.relation("address","district"),
        QueryFunc.relation("address","address"),
        QueryFunc.relation("address","postal_code"),
    ],
    join_table=["address","city","country"],
    join_on=[
        {
            QueryFunc.relation("store","address_id"):QueryFunc.relation("address","address_id")
        },
        {
            QueryFunc.relation("address","city_id"):QueryFunc.relation("city","city_id")
        },
        {
            QueryFunc.relation("city","country_id"):QueryFunc.relation("country","country_id")
        }
    ],
    order_by=["store_id"],
)
# Print sql query:
print(client.parse_query(query))
# Execute the query:
client.read_query(query)

SELECT "store"."store_id","country"."country","city"."city","address"."district","address"."address","address"."postal_code" FROM "store" INNER JOIN "address" ON ("store"."address_id" = "address"."address_id") INNER JOIN "city" ON ("address"."city_id" = "city"."city_id") INNER JOIN "country" ON ("city"."country_id" = "country"."country_id") ORDER BY "store_id" ASC


,store_id,country,city,district,address,postal_code
0,1,Canada,Lethbridge,Alberta,47 MySakila Drive,
1,2,Australia,Woodridge,QLD,28 MySQL Boulevard,


Count rows in a table:

In [55]:
# Count number of rows in customer:
client.count("customer")

599

Execute a raw sql query:

In [58]:
# Get first 10 actors by last name:
client.read_sql_query("SELECT actor_id, first_name|| ' ' || last_name AS name FROM actor ORDER BY last_name LIMIT 10")

,actor_id,name
0,58,Christian Akroyd
1,92,Kirsten Akroyd
2,182,Debbie Akroyd
3,118,Cuba Allen
4,145,Kim Allen
5,194,Meryl Allen
6,76,Angelina Astaire
7,112,Russell Bacall
8,67,Jessica Bailey
9,190,Audrey Bailey


Use 'aliases' to change the name of columns:

In [66]:
# Get 10 'customer ids' where the mean payment amount is greater than 5 dollars:
q = client.build_query(
    "payment",
    projection=[
        QueryFunc.alias("customer_id", "Customer Id"),
        QueryFunc.avg("amount", alias="Mean payment amount")
    ],
    group_by=["customer_id"],
    order_by=["Mean payment amount"],
    having_filters={QueryFunc.avg("amount"):QueryOp.greater_than(5)},
    ascending=False,
    limit=10
)
print(client.parse_query(q))
df = client.read_query(q)
df

SELECT "customer_id" AS "Customer Id",AVG("amount") AS "Mean payment amount" FROM "payment" GROUP BY "customer_id" HAVING (AVG("amount") > 5) ORDER BY "Mean payment amount" DESC LIMIT 10


,Customer Id,Mean payment amount
0,187,5.619630
1,321,5.519412
2,19,5.490000
3,3,5.448333
4,311,5.390000
5,542,5.302500
6,310,5.297692
7,508,5.294348
8,259,5.156667
9,293,5.132857


Use a literal sql value when building a query, this is useful for more 'complex' select statements:

In [68]:
# Import 'Literal'
from hero_db_utils.utils.dtypes import Literal

q = client.build_query(
    "customer",
    projection=[
        "customer_id",
        QueryFunc.alias(Literal("first_name || ' ' || last_name"), "customer_name"), # Raw SQL
        QueryFunc.alias(
            Literal("CONCAT('customer for ', EXTRACT(YEAR FROM AGE(create_date)), ' years')"),  # Raw SQL.
            "customer_years"
        )
    ],
    distinct=True,
    limit=5
)
print(client.parse_query(q))
client.read_query(q)

SELECT DISTINCT "customer_id",first_name || ' ' || last_name AS "customer_name",CONCAT('customer for ', EXTRACT(YEAR FROM AGE(create_date)), ' years') AS "customer_years" FROM "customer" LIMIT 5


,customer_id,customer_name,customer_years
0,195,Vanessa Sims,customer for 15 years
1,439,Alexander Fennell,customer for 15 years
2,318,Brian Wyman,customer for 15 years
3,416,Jeffery Pinson,customer for 15 years
4,313,Donald Mahon,customer for 15 years


Initiate a client with named arguments:

In [76]:
# Set arguments manually:
client = PostgresDatabaseClient(
    db_name="dvd_rentals",
    db_username="hero",
    db_password="hero@2020",
    db_host="localhost",
    db_port="5432",
)

Filter with an OR statement:

In [88]:
from hero_db_utils.queries.postgres import QueryOperation

# Get films where the category id is is one of the following and the rating is not 'R':
film_categories_ids = {
    1:"Animation",
    3:"Children",
    8:"Family",
    10:"Games"
}
# Create OR statement:
filter = QueryOperation.q_or(values=film_categories_ids.keys(), key="category_id")
# Add an AND clause to the filter:
filter.join_and(QueryOp.not_equals("R").resolve("rating"))
# Build query using the filter:
query = client.build_query(
    "film",
    ["title","description","rating","category_id"],
    join_table="film_category",
    join_on={QueryFunc.relation("film","film_id"):QueryFunc.relation("film_category","film_id")},
    filters=filter,
    limit=10
)
# Read query
print(client.parse_query(query))
client.read_query(query)

SELECT "title","description","rating","category_id" FROM "film" INNER JOIN "film_category" ON ("film"."film_id" = "film_category"."film_id") WHERE (("category_id" = 1) OR ("category_id" = 3) OR ("category_id" = 8) OR ("category_id" = 10)) AND ("rating" <> 'R') LIMIT 10


,title,description,rating,category_id
0,African Egg,A Fast-Paced Documentary of a Pastry Chef And a Dentist who must Pursue a Forensic Psychologist in The Gulf of Mexico,G,8
1,Amadeus Holy,A Emotional Display of a Pioneer And a Technical Writer who must Battle a Man in A Baloon,PG,1
2,Antitrust Tomatoes,A Fateful Yarn of a Womanizer And a Feminist who must Succumb a Database Administrator in Ancient India,NC-17,1
3,Apache Divine,A Awe-Inspiring Reflection of a Pastry Chef And a Teacher who must Overcome a Sumo Wrestler in A U-Boat,NC-17,8
4,Ark Ridgemont,A Beautiful Yarn of a Pioneer And a Monkey who must Pursue a Explorer in The Sahara Desert,NC-17,1
5,Atlantis Cause,A Thrilling Yarn of a Feminist And a Hunter who must Fight a Technical Writer in A Shark Tank,G,8
6,Autumn Crow,A Beautiful Tale of a Dentist And a Mad Cow who must Battle a Moose in The Sahara Desert,G,10
7,Backlash Undefeated,A Stunning Character Study of a Mad Scientist And a Mad Cow who must Kill a Car in A Monastery,PG-13,3
8,Baked Cleopatra,A Stunning Drama of a Forensic Psychologist And a Husband who must Overcome a Waitress in A Monastery,G,8
9,Bang Kwai,A Epic Drama of a Madman And a Cat who must Face a A Shark in An Abandoned Amusement Park,NC-17,8


Modify a query object after creating it:

In [90]:
query = client.build_query("film", ["film_id","title"], limit=3, order_by=["film_id"])
print(client.parse_query(query))
client.read_query(query)

SELECT "film_id","title" FROM "film" ORDER BY "film_id" ASC LIMIT 3


,film_id,title
0,1,Academy Dinosaur
1,2,Ace Goldfinger
2,3,Adaptation Holes


In [91]:
# Retrieve next 2 rows:
query = query.offset(3).limit(2)
query.resolve()
print(client.parse_query(query))
client.read_query(query)

SELECT "film_id","title" FROM "film" ORDER BY "film_id" ASC OFFSET 3 ROWS LIMIT 2


,film_id,title
0,4,Affair Prejudice
1,5,African Egg
